In [1]:
!pip install stable-baselines3[extra]
!pip install "autorom[accept-rom-license]"


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1

In [2]:
!pip install gymnasium>=1.0.0 ale-py>=0.9.0 shimmy>=1.0.0


In [3]:
import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

# Force uninstall and reinstall numpy
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4"])

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
kaggle-environments 1.18.0 requires shimmy>=1.2.1, but you have shimmy 1.1.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
goog

0

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

import ale_py
import gymnasium as gym
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import numpy as np
import random
import torch
import gc

In [5]:
gc.collect()
torch.cuda.empty_cache()

In [6]:
CONFIG_ID = 2

configs = [
    {"lr": 0.00035, "gamma": 0.99,  "batch": 12, "eps_start": 1.0, "eps_end": 0.03, "eps_decay": 0.25},
    {"lr": 0.00075, "gamma": 0.97,  "batch": 20, "eps_start": 1.0, "eps_end": 0.07, "eps_decay": 0.15},
    {"lr": 0.00008, "gamma": 0.992, "batch": 10, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.3},
    {"lr": 0.00045, "gamma": 0.985, "batch": 18, "eps_start": 1.0, "eps_end": 0.04, "eps_decay": 0.2},
    {"lr": 0.00028, "gamma": 0.98,  "batch": 14, "eps_start": 1.0, "eps_end": 0.015, "eps_decay": 0.18},
    {"lr": 0.00055, "gamma": 0.99,  "batch": 10, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.22},
    {"lr": 0.0009,  "gamma": 0.975, "batch": 12, "eps_start": 1.0, "eps_end": 0.08, "eps_decay": 0.12},
    {"lr": 0.00012, "gamma": 0.996, "batch": 16, "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.28},
    {"lr": 0.0004,  "gamma": 0.988, "batch": 15, "eps_start": 1.0, "eps_end": 0.035, "eps_decay": 0.19},
    {"lr": 0.00018, "gamma": 0.993, "batch": 13, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.16},
]


cfg = configs[CONFIG_ID - 1]


print(f"Policy: CnnPolicy")
print(f"Hyperparameters:")
print(f"  • Learning Rate (lr): {cfg['lr']}")
print(f"  • Gamma (γ): {cfg['gamma']}")
print(f"  • Batch Size: {cfg['batch']}")
print(f"  • Epsilon Start: {cfg['eps_start']}")
print(f"  • Epsilon End: {cfg['eps_end']}")
print(f"  • Epsilon Decay: {cfg['eps_decay']}")

Policy: CnnPolicy
Hyperparameters:
  • Learning Rate (lr): 0.00075
  • Gamma (γ): 0.97
  • Batch Size: 20
  • Epsilon Start: 1.0
  • Epsilon End: 0.07
  • Epsilon Decay: 0.15


In [7]:
env_test = gym.make("ALE/SpaceInvaders-v5")
random_scores = []

for episode in range(1, 21):
    obs, _ = env_test.reset()
    done = False
    truncated = False
    score = 0
    while not (done or truncated):
        action = random.choice([0, 1, 2, 3, 4, 5])
        obs, reward, done, truncated, _ = env_test.step(action)
        score += reward
    random_scores.append(score)
    if episode % 5 == 0:
        print(f"  Episode {episode}: {score}")

avg_random_score = np.mean(random_scores)
print(f"\nRandom Agent Average Score: {avg_random_score:.2f}")
env_test.close()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


  Episode 5: 100.0
  Episode 10: 80.0
  Episode 15: 190.0
  Episode 20: 225.0

Random Agent Average Score: 145.00


In [8]:
env = gym.make("ALE/SpaceInvaders-v5", frameskip=4)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)

In [9]:
model = DQN(
    "CnnPolicy",
    env,
    learning_rate=cfg["lr"],
    gamma=cfg["gamma"],
    batch_size=cfg["batch"],
    buffer_size=5000,
    learning_starts=2000,
    exploration_initial_eps=cfg["eps_start"],
    exploration_final_eps=cfg["eps_end"],
    exploration_fraction=cfg["eps_decay"],
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    tensorboard_log=f"./logs/",
    verbose=1,
    device="cuda"
)

print("Model created successfully!")
print(f"   Buffer size: {model.buffer_size:,}")
print(f"   Learning starts at: {model.learning_starts:,} steps")
print(f"   Policy: CnnPolicy (Convolutional Neural Network)")

print(f"\nInitial GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

Using cuda device
Wrapping the env in a VecTransposeImage.
Model created successfully!
   Buffer size: 5,000
   Learning starts at: 2,000 steps
   Policy: CnnPolicy (Convolutional Neural Network)

Initial GPU memory: 0.09 GB


In [11]:
try:
    model.learn(
        total_timesteps=100_000,
        tb_log_name=f"SpaceInvaders_Config{CONFIG_ID}_CnnPolicy",
        log_interval=1000,
        progress_bar=True
    )

    print("\nTraining completed successfully!")
    print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print(f"\nOUT OF MEMORY ERROR")
        print(f"Current settings: buffer={model.buffer_size}, batch={cfg['batch']}")
        print(f"\nEMERGENCY FIX:")
        print(f"   1. Change buffer_size=5000")
        print(f"   2. Change batch_size=4")
        print(f"   3. Or use device='cpu' (slower but stable)")
    raise e

Logging to ./logs/SpaceInvaders_Config2_CnnPolicy_1


Output()


Training completed successfully!
Max GPU memory used: 0.31 GB


In [14]:
mean_reward, std_reward = evaluate_policy(
    model,
    env,
    n_eval_episodes=10,
    deterministic=True
)

print(f"\nResults over 10 episodes:")
print(f"   Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"   Random Baseline: {avg_random_score:.2f}")
print(f"   Improvement: {mean_reward - avg_random_score:.2f}")


Results over 10 episodes:
   Mean Reward: 190.50 ± 156.34
   Random Baseline: 145.00
   Improvement: 45.50


In [15]:
model_name = "dqn_model_2"
model.save(model_name)

backup_name = f"dqn_spaceinvaders_exp{CONFIG_ID}"
model.save(backup_name)

print(f"\nModels saved:")
print(f"   • {model_name}.zip (required submission)")
print(f"   • {backup_name}.zip (backup with config ID)")


Models saved:
   • dqn_model_2.zip (required submission)
   • dqn_spaceinvaders_exp2.zip (backup with config ID)
